<a href="https://colab.research.google.com/github/fduque/kaggle_PredictingHousePrices_ML/blob/master/predicting_HousePrices_kaggle_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd

In [0]:
data = pd.read_csv("train.csv")
#defining attr,feat
X = data.drop(["Id","SalePrice"], axis=1)
#defining label
Y = data['SalePrice']

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
from sklearn.model_selection import train_test_split
#defining validation and trainning dataset
Xtr, Xval, ytr, yval = train_test_split(X,Y,train_size=0.5, random_state=0)

Xtr.shape, Xval.shape, ytr.shape, yval.shape

((730, 79), (730, 79), (730,), (730,))

In [5]:
Xtr.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
733,20,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,5,6,1961,1983,Hip,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,594,Unf,0,270,864,GasA,Ex,Y,SBrkr,1144,0,0,1144,1,0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1961.0,RFn,1,264,TA,TA,Y,165,0,0,0,0,0,NaN,GdWo,Shed,400,3,2009,WD,Normal
382,60,RL,79.0,9245,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Av,Unf,0,Unf,0,939,939,GasA,Ex,Y,SBrkr,939,858,0,1797,0,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2006.0,RFn,2,639,TA,TA,Y,144,53,0,0,0,0,NaN,NaN,NaN,0,4,2007,WD,Normal
1232,90,RL,70.0,9842,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,NAmes,Norm,Norm,Duplex,1Story,4,5,1962,1962,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,TA,Y,SBrkr,1224,0,0,1224,0,0,2,0,2,2,TA,6,Typ,0,NaN,CarPort,1962.0,Unf,2,462,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal
1354,60,RL,NaN,10316,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,735,Unf,0,257,992,GasA,Ex,Y,SBrkr,992,873,0,1865,1,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2000.0,RFn,3,839,TA,TA,Y,0,184,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
331,20,RL,70.0,8176,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1958,1992,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,846,Unf,0,210,1056,GasA,Fa,Y,SBrkr,1056,0,0,1056,1,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1958.0,RFn,1,308,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal


In [0]:
#as we have seen above, there are numbers of categorical variables. So, we need to convert these text variable in numbers. 
#to do it so, we can use one hot encoding...
#lets defining the columns to be transformed
cat_cols = ['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']



In [7]:
pip install category_encoders

In [0]:
#encoding...
from category_encoders import OneHotEncoder
#handle_missing=0 means that if exist any missing value, like null, define it = Zero
cat_encoder = OneHotEncoder(cols=cat_cols, drop_invariant=True, handle_missing=0)
#encoding trainning data...
Xtr = cat_encoder.fit_transform(Xtr)
#encoding validation data...
Xval = cat_encoder.transform(Xval)

In [0]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
#encoding trainning data...
Xtr = imputer.fit_transform(Xtr)
Xval = imputer.transform(Xval)


In [10]:
Xtr

array([[ 20.,   1.,   0., ...,   0.,   0.,   0.],
       [ 60.,   1.,   0., ...,   0.,   0.,   0.],
       [ 90.,   1.,   0., ...,   0.,   0.,   0.],
       ...,
       [ 90.,   0.,   1., ...,   0.,   0.,   0.],
       [120.,   1.,   0., ...,   0.,   0.,   0.],
       [ 60.,   1.,   0., ...,   0.,   0.,   0.]])

In [0]:
from tensorflow import keras as tfk

In [12]:
#defining standard seed to guarantee the reproduceability of model
tf.random.set_seed(2)

#first layer, input data
inp = tfk.layers.Input((Xtr.shape[1], ))

#hidden layers
# 100 = qty of neurons
# relu function: x = w1*f1 + w2*f2 > max(0, x)
#at the end, declare "inp" that means the layer input, the layer that comes immediately before
hid = tfk.layers.Dense(100,activation='relu')(inp)

#output layer
# 1, meas just one output layer
# activation Linear = its the simplest way, which x = w1*f1 + w2*f2
#at the end, declare "hid" that means the layer hidden
out = tfk.layers.Dense(1, activation='linear')(hid)

#1st par = input layer
#2nd par = output layer
mdl = tfk.Model(inp, out)
#loss msle = minimum square logarithm error
#defining the optimizer = Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
mdl.compile(loss='msle', optimizer='adam')
#print how our neural model is organized
mdl.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               28600     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 28,701
Trainable params: 28,701
Non-trainable params: 0
_________________________________________________________________


**Tranning - Version 1**

In [13]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1)

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7781 - val_loss: 144.6273
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7779 - val_loss: 144.6273
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7779 - val_loss: 144.6273
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 144.7780 - val_loss: 144.6273
Epoch 10/10
730/730 [==============================] - 2s 2ms/st

In [14]:
#High Loss rate! 
import numpy as np
loss1 = np.sqrt(144.7788)
print("Model 01 loss rate:", loss1,"%")

Model 01 loss rate: 12.032406243141892 %


In [0]:
#High Loss rate! 
#One of the reasons, some features, such as loss_area, have a wide range of values. 
#Reducing this range, through out Scalling method, can improve our model
#Lets do it on Trainning Version2


**Trainning - Version 2 **

Applying scalling ...

In [0]:
#re-doing inputing process...
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
#encoding trainning data...
Xtr = imputer.fit_transform(Xtr)
Xval = imputer.transform(Xval)

In [0]:
#applying scalling method...
#normally, instead of using MaxAbsScaller, we would use StandardScaler (subtract the mean and divide by the standard deviation)
#in cases of very sparse datasets, like this one,StandardScaler it is not indicated
#We choose to use MaxAbsScaller, better for this model, MaxAbsScaller(get the higher absolute value of the column and divide each value by it)
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
Xtr = scaler.fit_transform(Xtr)
Xval = scaler.transform(Xval)

In [17]:
Xtr

array([[0.10526316, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31578947, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.47368421, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.47368421, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.63157895, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31578947, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
from tensorflow import keras as tfk

In [19]:
#defining standard seed to guarantee the reproduceability of model
tf.random.set_seed(2)

#first layer, input data
inp = tfk.layers.Input((Xtr.shape[1], ))

#hidden layers
# 100 = qty of neurons
# relu function: x = w1*f1 + w2*f2 > max(0, x)
#at the end, declare "inp" that means the layer input, the layer that comes immediately before
hid = tfk.layers.Dense(100,activation='relu')(inp)

#output layer
# 1, meas just one output layer
# activation Linear = its the simplest way, which x = w1*f1 + w2*f2
#at the end, declare "hid" that means the layer hidden
out = tfk.layers.Dense(1, activation='linear')(hid)

#1st par = input layer
#2nd par = output layer
mdl = tfk.Model(inp, out)
#loss msle = minimum square logarithm error
#defining the optimizer = Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
mdl.compile(loss='msle', optimizer='adam')
#print how our neural model is organized
mdl.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               28600     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 28,701
Trainable params: 28,701
Non-trainable params: 0
_________________________________________________________________


In [20]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1)

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 59.0277 - val_loss: 39.0566
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 33.2110 - val_loss: 28.6764
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 25.6345 - val_loss: 22.9695
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 20.9052 - val_loss: 19.0063
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 17.4412 - val_loss: 15.9630
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 14.7064 - val_loss: 13.4994
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 12.4548 - val_loss: 11.4408
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 10.5558 - val_loss: 9.6898
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 8.9330 - val_loss: 8.1874
Epoch 10/10
730/730 [==============================] - 2s 2ms/step - loss: 7.5361 - v

In [21]:
#The model has improved! 
import numpy as np
loss2 = np.sqrt(7.53)
print("Model 02 loss rate:", loss2,"%")

Model 02 loss rate: 2.7440845468024486 %


Trainning 3 
Change activation function to Relu

In [22]:
#defining standard seed to guarantee the reproduceability of model
tf.random.set_seed(2)

#first layer, input data
inp = tfk.layers.Input((Xtr.shape[1], ))

#hidden layers
# 100 = qty of neurons
# relu function: x = w1*f1 + w2*f2 > max(0, x)
#at the end, declare "inp" that means the layer input, the layer that comes immediately before
hid = tfk.layers.Dense(100,activation='relu')(inp)

#output layer
# 1, meas just one output layer
# activation Linear = its the simplest way, which x = w1*f1 + w2*f2
#at the end, declare "hid" that means the layer hidden
out = tfk.layers.Dense(1, activation='relu')(hid)

#1st par = input layer
#2nd par = output layer
mdl = tfk.Model(inp, out)
#loss msle = minimum square logarithm error
#defining the optimizer = Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
mdl.compile(loss='msle', optimizer='adam')
#print how our neural model is organized
mdl.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               28600     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 28,701
Trainable params: 28,701
Non-trainable params: 0
_________________________________________________________________


In [23]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1)

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 59.0277 - val_loss: 39.0566
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 33.2110 - val_loss: 28.6764
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 25.6345 - val_loss: 22.9695
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 20.9052 - val_loss: 19.0063
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 17.4412 - val_loss: 15.9630
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 14.7064 - val_loss: 13.4994
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 12.4548 - val_loss: 11.4408
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 10.5558 - val_loss: 9.6898
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 8.9330 - val_loss: 8.1874
Epoch 10/10
730/730 [==============================] - 2s 2ms/step - loss: 7.5361 - v

In [24]:
#Nothing has changed! 
import numpy as np
loss3 = np.sqrt(7.53)
print("Model 03 loss rate:", loss3,"%")

Model 03 loss rate: 2.7440845468024486 %


Trainning 4
Increasing one hidden layer (Now we're deep!:))

In [25]:
#defining standard seed to guarantee the reproduceability of model
tf.random.set_seed(2)

#first layer, input data
inp = tfk.layers.Input((Xtr.shape[1], ))

#hidden layers
# 100 = qty of neurons
# relu function: x = w1*f1 + w2*f2 > max(0, x)
#at the end, declare "inp" that means the layer input, the layer that comes immediately before
hid1 = tfk.layers.Dense(100,activation='relu')(inp)

#two hidden layers, its considered a deep learning model...
hid2 = tfk.layers.Dense(100,activation='relu')(hid1)

#output layer
# 1, meas just one output layer
# activation Linear = its the simplest way, which x = w1*f1 + w2*f2
#at the end, declare "hid" that means the layer hidden
out = tfk.layers.Dense(1, activation='linear')(hid2)

#1st par = input layer
#2nd par = output layer
mdl = tfk.Model(inp, out)
#loss msle = minimum square logarithm error
#defining the optimizer = Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
mdl.compile(loss='msle', optimizer='adam')
#print how our neural model is organized
mdl.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               28600     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 38,801
Trainable params: 38,801
Non-trainable params: 0
_________________________________________________________________


In [26]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1)

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 36.4788 - val_loss: 11.5034
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 7.4209 - val_loss: 4.7114
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 3.3309 - val_loss: 2.2611
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 1.6266 - val_loss: 1.1114
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 0.8013 - val_loss: 0.5446
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 0.4010 - val_loss: 0.2796
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 0.2208 - val_loss: 0.1680
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1497 - val_loss: 0.1278
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1251 - val_loss: 0.1138
Epoch 10/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1143 - val_loss: 0.10

In [27]:
#The model has improved! 
import numpy as np
loss4 = np.sqrt(0.1143)
print("Model 04 loss rate:", loss4,"%")

Model 04 loss rate: 0.33808283008753937 %


Trainning - Version 5
Applying Early Stoppping (



In [0]:

#EarlyStopping: great resource that helps to stop run new epochs if the loss rate is becoming worse

#Example: If your define 100 epochs, but after run epoch number 55 the loss rate is becaming worse, the function run the number of patience defined and stop

#patience = qty of iterations to be executed after the loss rate has stopped reducing
#patience = number of steps that the model can make without reducing the Error
#patiente = hepls to find out the point in the function with min error
#min_delta = define the value that informs patient to stop or not
#val_loss = inform function has to be considered, trainning or validation loss
#mode = informs that the function has to be minimized or max
#restore_best_weights = restore best weight values found or not
es = tfk.callbacks.EarlyStopping(
    monitor = 'val_loss', min_delta=0,patience=10,mode='min',restore_best_weights=True
)

In [29]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
#including callbacks = 
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1, callback=[es])

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1052 - val_loss: 0.0960
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0954 - val_loss: 0.0862
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0850 - val_loss: 0.0757
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0740 - val_loss: 0.0658
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0636 - val_loss: 0.0569
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0547 - val_loss: 0.0487
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0473 - val_loss: 0.0432
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0419 - val_loss: 0.0396
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0379 - val_loss: 0.0369
Epoch 10/10
730/730 [==============================] - 2s 2ms/step - loss: 0.0349 - val_loss: 0.0348

In [30]:
#The model has improved! 
import numpy as np
loss5 = np.sqrt(0.0349)
print("Model 05 loss rate:", loss5,"%")

Model 05 loss rate: 0.18681541692269404 %


Trainning - Version 6 
Applying Dropout layer

- kind of regularization method
- change some previous layer values to ZERO
- helps to avoid overfitting
- helps the model not memorize the weights
- kind of hyper parameter

In [31]:
#defining standard seed to guarantee the reproduceability of model
tf.random.set_seed(2)

#first layer, input data
inp = tfk.layers.Input((Xtr.shape[1], ))

#hidden layers
# 100 = qty of neurons
# relu function: x = w1*f1 + w2*f2 > max(0, x)
#at the end, declare "inp" that means the layer input, the layer that comes immediately before
hid1 = tfk.layers.Dense(100,activation='relu')(inp)

#new layout, dropout layer
drop = tfk.layers.Dropout(0.5)(hid1)

#two hidden layers, its considered a deep learning model...
hid2 = tfk.layers.Dense(100,activation='relu')(drop)

#output layer
# 1, meas just one output layer
# activation Linear = its the simplest way, which x = w1*f1 + w2*f2
#at the end, declare "hid" that means the layer hidden
out = tfk.layers.Dense(1, activation='linear')(hid2)

#1st par = input layer
#2nd par = output layer
mdl = tfk.Model(inp, out)
#loss msle = minimum square logarithm error
#defining the optimizer = Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
mdl.compile(loss='msle', optimizer='adam')
#print how our neural model is organized
mdl.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               28600     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 38,801
Trainable params: 38,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
#EarlyStopping: great resource that helps to stop run new epochs if the loss rate is becoming worse

#Example: If your define 100 epochs, but after run epoch number 55 the loss rate is becaming worse, the function run the number of patience defined and stop

#patience = qty of iterations to be executed after the loss rate has stopped reducing
#patience = number of steps that the model can make without reducing the Error
#patiente = hepls to find out the point in the function with min error
#min_delta = define the value that informs patient to stop or not
#val_loss = inform function has to be considered, trainning or validation loss
#mode = informs that the function has to be minimized or max
#restore_best_weights = restore best weight values found or not
es = tfk.callbacks.EarlyStopping(
    monitor = 'val_loss', min_delta=0,patience=10,mode='min',restore_best_weights=True
)

In [33]:
#epoch = when the neural network has
#shuffle  = true : means that the function is going to sort again, giving some ramdoness for each round
#including callbacks = 
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=10, shuffle=True, batch_size=1, callback=[es])

Epoch 1/10
730/730 [==============================] - 2s 2ms/step - loss: 41.7158 - val_loss: 16.1529
Epoch 2/10
730/730 [==============================] - 2s 2ms/step - loss: 10.5287 - val_loss: 6.6995
Epoch 3/10
730/730 [==============================] - 2s 2ms/step - loss: 5.0576 - val_loss: 3.4426
Epoch 4/10
730/730 [==============================] - 2s 2ms/step - loss: 2.7167 - val_loss: 1.8022
Epoch 5/10
730/730 [==============================] - 2s 2ms/step - loss: 1.4568 - val_loss: 0.9248
Epoch 6/10
730/730 [==============================] - 2s 2ms/step - loss: 0.8137 - val_loss: 0.4673
Epoch 7/10
730/730 [==============================] - 2s 2ms/step - loss: 0.4535 - val_loss: 0.2463
Epoch 8/10
730/730 [==============================] - 2s 2ms/step - loss: 0.2684 - val_loss: 0.1517
Epoch 9/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1991 - val_loss: 0.1171
Epoch 10/10
730/730 [==============================] - 2s 2ms/step - loss: 0.1995 - val_loss: 0.1

In [34]:
#The model has improved! 
import numpy as np
loss6 = np.sqrt(0.1995)
print("Model 06 loss rate:", loss6,"%")

Model 06 loss rate: 0.4466542286825459 %


***Journey Summary***



In [35]:

#Summary
print("Model 01 loss rate:", round(loss1,2),"%")
print("Model 02 - Applying scalling - loss rate:",  round(loss2,2),"%")
print("Model 03 - Change activation function to Relu, Output Layer - loss rate:",  round(loss3,2),"%")
print("Model 04 - Increasing one hidden layer - loss rate:",  round(loss4,2),"%")
print("Model 05 - Applying Early Stoppping - loss rate:",  round(loss5,2),"%")
print("Model 06 - Applying Dropout layer - loss rate:",  round(loss6,2),"%")

Model 01 loss rate: 12.03 %
Model 02 - Applying scalling - loss rate: 2.74 %
Model 03 - Change activation function to Relu, Output Layer - loss rate: 2.74 %
Model 04 - Increasing one hidden layer - loss rate: 0.34 %
Model 05 - Applying Early Stoppping - loss rate: 0.19 %
Model 06 - Applying Dropout layer - loss rate: 0.45 %


Credits: Mario Filho - https://github.com/ledmaster